# <span style='color:SteelBlue'>PB6 - Classification automatique de produits (part 4)</span>

<img src="https://logo-marque.com/wp-content/uploads/2020/11/Flipkart-Logo.png" width="300" height="200">

# <span style='background:white; color:black'>Sommaire</span>

**Introduction**

Importations des librairies<br>

Paramètres d'affichage<br>

Fonctions <br>

**Etape 1 : Rappel RGPD**

**Etape 2 : Collecte des données**
- _1/ Stratégie de collecte_
- _2/ API disponible_

**Etape 3 : Requête vers API**

**Etape 4 : Filtrage des informations cibles**

**Etape 5 : Export CSV**

**Conclusion**

# <span style='background:blue'>Introduction</span>

L'entreprise **"Flipkart"** est une entreprise indienne qui souhaite lancer une
marketplace e-commerce. Sur cette place de marché anglophone, des vendeurs
proposent des articles à des acheteurs en postant une photo et une description.
Pour rendre l’expérience utilisateur des vendeurs (faciliter la mise en ligne
de nouveaux articles) et des acheteurs (faciliter la recherche de produits) 
la plus fluide possible, et dans l'optique d'un passage à l'échelle, 
il devient nécessaire **d'automatiser l'attribution de la catégorie** d'un 
article (actuellement effectuée manuellement par les vendeurs, donc peu 
fiable).
___
**Missions**<br>
1/ Faire une **étude de la faisabilité d’un moteur de classification**
d’article basé sur une **description** pour automatiser l’attribution
de catégorie de l’article. 

2/ Faire une **étude de la faisabilité d’un moteur de classification**
d’article basé sur une **image** pour automatiser l’attribution
de catégorie de l’article. 

3/ **Réaliser une classification supervisée** à partir des images par la mise
en place d’une data augmentation afin d’optimiser le modèle. 

4/ **Extraire les 10 premiers produits à base de « champagne »** issus de
l’API fournie dans un fichier “.csv”, contenant pour chaque produit les
données suivantes : foodId, label, category, foodContentsLabel, image.
___
Ce notebook presente la 4ème partie concernant l'extraction des 10 premiers 
produits dans l'API. 

# <span style='background:grey'>Importations des librairies utilisees dans ce notebook</span>

In [1]:
# Librairies generales
import sys
import time
from datetime import datetime
import csv

# Librairies data science
import pandas as pd

# API
import requests
import json
from colored import fg, attr

# Affichage plot dans le notebook
%matplotlib inline

# Versions
print("Version des librairies utilisees :")
print("Python        : " + sys.version)
print("Csv           : " + csv.__version__)
print("Json          : " + json.__version__)
print("Pandas        : " + pd.__version__)
print("Requests      : " + requests.__version__)

# Afficher heure lancement
maintenant = datetime.now().isoformat()
print("\nCode lance le : " + maintenant)

# Enregistrer l'heure de debut
start_time = time.time()

# Activer la verification PEP8
print("")
%load_ext pycodestyle_magic
%pycodestyle_on

Version des librairies utilisees :
Python        : 3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC v.1937 64 bit (AMD64)]
Csv           : 1.0
Json          : 2.0.9
Pandas        : 2.1.4
Requests      : 2.31.0

Code lance le : 2024-02-07T18:06:13.072195



# <span style='background:grey'>Paramètres d'affichage</span>

In [2]:
# parametrage des paramètres d'affichage dans le notebook
pd.set_option("display.max_row", 20)
pd.set_option("display.max_column", 10)
pd.set_option("display.precision", 2)

# <span style='background:grey'>Fonctions</span>

In [3]:
def calculer_duree_notebook(start_time):
    """
    Cette procédure calcule et affiche la durée totale depuis un temps de
    début donné.

    Args:
        start_time (float): Le temps de début en secondes depuis l'époque.

    Returns:
        None
    """
    # Calculer la durée totale
    total_time = time.time() - start_time

    # Convertir en minutes et secondes
    minutes, seconds = divmod(total_time, 60)

    # Afficher la durée totale
    message = (
        "Durée totale pour faire tourner le notebook : "
        "{0} minute(s) {1} seconde(s)"
    )
    print(message.format(int(minutes), int(seconds)))

# <span style='background:blue'>Etape 1 : Rappel RGPD</span>

Voici les 5 grand principe RGPD (Règlement sur la Protection des Données Personnelles) : 

1. **Finalité** : Les données sont collectées pour un but bien déterminé et légitime et ne sont pas traitées ultérieurement de façon incompatible avec cet objectif initial. 
   
2. **Pertinence** : Seules les données strictement nécessaires à la réalisation de l’objectif poursuivi doivent être collectées. Il s’agit donc de minimiser autant que possible la collecte des données.

3. **Durée limitée de conservation** : Les données ne doivent être conservées sous une forme identifiante et en « base active » que le temps nécessaire à la réalisation de l’objectif poursuivi et doivent être par la suite détruites, anonymisées ou archivées dans le respect des obligations légales applicables en matière de conservation des archives publiques.

4. **Sécurité** : Le responsable de traitement de la collectivité doit prendre toutes les mesures utiles pour garantir l’intégrité et la confidentialité de ces données, en s’assurant notamment que des tiers non autorisés n’y auront pas accès. 

5. **Droits des personnes** : Les personnes concernées par les traitements doivent conserver la maitrise des données qui les concernent. Ces droits « Informatique et Libertés », qu’elles peuvent exercer auprès de la collectivité qui détient ces informations sont :
    - le droit d’accéder à leurs données et d’en obtenir une copie.
    - le droit de les rectifier.
    - le droit de s’opposer à leur utilisation, sauf si le traitement répond à une obligation légale.

        *La loi pour une République numérique d’octobre 2016 est venue renforcer ces droits en prévoyant notamment la possibilité pour les personnes concernées de les exercer par voie électronique, ainsi que de donner des directives relatives à la conservation, à l'effacement et à la communication de leurs données après leur décès.*

Source : [CNIL](https://www.cnil.fr/fr/collectivites-territoriales/les-principes-cles-de-la-protection-des-donnees)

___

Pour la collecte d'information depuis l'API, le recueil des données est effectué en respect des principes de **finalité** et de **pertinence**.

En effet, nous déclarons la **finalité** de notre collecte de données à l'API qui est de recencer une liste de produits à base de "champagne" pour une élargir la gamme de produits à l'épicerie fine.

Egalement, nous  **minimisons** la collecte des données en ne recueillant que les 10 premiers produits à base de "champagne" dans l'API pour commencer notre travail.

# <span style='background:blue'>Etape 2 : Collecte des données</span>

## <span style='background:green'>1/ Stratégie de collecte</span>

Notre client souhaite élargir sa gamme de produits à l'épicierie fine. Pour cela, il a besoin de connaître les 10 premiers produits à base de "champagne" issus de l'API fournie : https://rapidapi.com/edamam/api/edamam-food-and-grocery-database

Nous choississons de collecter les données grâce à script **python** en utilisant le module **requests**  qui est une bibliothèque HTTP Python qui permet de faire des requêtes HTTP en utilisant Python.

Nous allons utiliser la méthode **GET** pour récupérer les données des produits à base de champagne de l'API puis nous filtrerons sur les 10 premiers produits et sur les informations qui nous intéressent.

## <span style='background:green'>2/ API disponible</span>

**Edanam** est une société qui propose plusieurs API pour obtenir des informations sur les produits alimentaires :

- **Nutrition Analysis API** : API qui fournit des informations détaillées sur la valeur nutritionnelle des aliments

- **Food and Grocery Database API** : API qui accède à une base de données étendue de produits alimentaires et d'épicerie

- **Recipe Search API** : API qui permet de rechercher des recettes en fonction de différents critères tels que les ingrédients, le type de plat, etc...

- **Recipe Content Management API** : API qui permet aux utilisateurs de gérer le contenu des recettes

- **Food Analytics Data** : API qui fournit des données analytiques sur les tendances alimentaires, les habitudes de consommation, etc...

- **Meal Planner API (beta)** : API, actuellement en version bêta, qui permet aux utilisateurs de planifier des repas en fonction de leurs préférences alimentaires, de leurs objectifs nutritionnels, etc...

- **Recipe Licencing** : API qui gère les licences de recettes

Le lien de l'API fournie par notre client concerne l'API **Food and Grocery Database API**.

# <span style='background:blue'>Etape 3 : Requête vers API</span>

Nous écrivons une requête pour obtenir les produits à base de "*champagne*" de l'API.

In [4]:
# URL de l'API
url = "https://api.edamam.com/api/food-database/v2/parser"

# Clé et ID de l'API
API_KEY = ''
API_ID = ''
# (volontairement effacés après utilisation pour des raisons de sécurité
# de ce notebook)

# Paramètres de la requête (ingrédient : champagne)
query_string = {"app_key": API_KEY, "app_id": API_ID, "ingr": "champagne"}

# Requête GET (réponse HTTP en format JSON) et transformation en dictionnaire
response_dict = requests.request("GET", url, params=query_string).json()

Nous explorons le contenu de la réponse de l'API pour comprendre sa structure

In [5]:
for key, value in response_dict.items():
    print(f"Clé: {key}")

Clé: text
Clé: parsed
Clé: hints
Clé: _links


La réponse est composéee de 4 éléments. Nous explorons un peu plus profondement le contenu de ces éléments pour mieux comprendre la structure des données. Nous affichons ainsi le contenu de chaque élément avec des couleurs et une indentation différente pour mieux visualiser la structure des données.

In [6]:
# Convertir le dictionnaire en JSON
response_json = json.dumps(response_dict, indent=4)

# Diviser la chaîne JSON en lignes
lines = response_json.split('\n')

# Définir une liste de couleurs
colors = [fg(226), fg(82), fg(196), fg(213), fg(226), fg(15)]

# Parcourir chaque ligne et colorier en fonction de l'indentation
for line in lines:
    indentation = len(line) - len(line.lstrip(' '))
    depth = indentation // 4
    color = colors[depth % len(colors)]
    print(color + line + attr(0))

{
    "text": "champagne",
    "parsed": [
        {
            "food": {
                "foodId": "food_a656mk2a5dmqb2adiamu6beihduu",
                "label": "Champagne",
                "knownAs": "dry white wine",
                "nutrients": {
                    "ENERC_KCAL": 82.0,
                    "PROCNT": 0.07,
                    "FAT": 0.0,
                    "CHOCDF": 2.6,
                    "FIBTG": 0.0
                },
                "category": "Generic foods",
                "categoryLabel": "food",
                "image": "https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg"
            }
        }
    ],
    "hints": [
        {
            "food": {
                "foodId": "food_a656mk2a5dmqb2adiamu6beihduu",
                "label": "Champagne",
                "knownAs": "dry white wine",
                "nutrients": {
                    "ENERC_KCAL": 82.0,
                    "PROCNT": 0.07,
                    "FAT": 0.0,
    

Voici une explication des éléments de la réponse de l'API :
- *text* : le texte à rechercher
- *parsed* : produit présentant le meilleur résultat de recherche
- *hints* : tous les produit de la recherche répondant à la recherche
- *_links* : liens vers la page suivante des résultats de la recherche


Le dictionnaire **hints** contient des informations pour chaque produit selon cette arborescence :
- food
    - foodId
    - label
    - knownAs
    - nutrients
    - category
    - categoryLabel
    - foodContentsLabel
    - image
- measure
    - uri
    - label
    - weight

*Remarque : certains produits ne contiennent pas toutes les informations.*

# <span style='background:blue'>Etape 4 : Filtrage des informations cibles</span>

Nous allons donc explorer le contenu de **hints** pour obtenir les informations recherchées pour les 10 premiers produits, à savoir : 
- **foodId** : identifiant du produit
- **label** : nom du produit
- **category** : catégorie du produit
- **foodContentsLabel** : ingrédients contenus dans le produit
- **image** : lien URL de l'image du produit

In [7]:
# Info à extraire
info_label = ["foodId", "label", "category", "foodContentsLabel", "image"]

# Liste vide pour stocker les produits extraits
produit_extracted_list = []

# Initialiser le compteur
compteur = 1

# SI la clé "hints" existe dans le dictionnaire
if "hints" in response_dict:

    # POUR chaque produit de la clé "hints"
    for produit_dict in response_dict["hints"]:
        print(f"Extraction produit n°{compteur}...")

        # Récupérer le dictionnaire "food" (dictionnaire vide si absent)
        food_dict = produit_dict.get('food', {})

        # Créer un dictionnaire avec les info du produit ('clef' : 'valeur')
        info_dict = {
            info_label: food_dict.get(info_label) for info_label in info_label
        }

        # Ajouter le dictionnaire à la liste des produits extraits
        produit_extracted_list.append(info_dict)

        # Incrémenter le compteur
        compteur += 1

        # SI le compteur atteint 10, sortir de la boucle
        if compteur > 10:
            break

Extraction produit n°1...
Extraction produit n°2...
Extraction produit n°3...
Extraction produit n°4...
Extraction produit n°5...
Extraction produit n°6...
Extraction produit n°7...
Extraction produit n°8...
Extraction produit n°9...
Extraction produit n°10...


# <span style='background:blue'>Etape 5 : Export CSV</span>

Nous enregistrons les informations receuillies dans un fichier **.csv**

In [8]:
# Nom du fichier
export_nom_fichier = 'berthe_pierrick_5_export_api_122023.csv'

# Ouvrir le fichier en mode écriture
with open(export_nom_fichier, 'w', newline='') as fichier_csv:

    # Création du writer
    writer = csv.DictWriter(fichier_csv, fieldnames=info_label)

    # Écrire les en-têtes
    writer.writeheader()

    # Écrire chaque ligne extraites
    for ligne in produit_extracted_list:
        writer.writerow(ligne)

Nous vérifions que le fichier a bien été créé et que les données sont bien enregistrées.

In [9]:
export_nom_fichier_df = pd.read_csv(export_nom_fichier, sep=",")
display(export_nom_fichier_df)

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
7,food_byap67hab6evc3a0f9w1oag3s0qf,Champagne Sorbet,Generic meals,Sugar; Lemon juice; brandy; Champagne; Peach,NaN
8,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
9,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN


Certaines cellules présentent des valeurs nulles (Nan) car la donnée n'était pas renseignée dans l'API.  

# <span style='background:blue'>Conclusion</span>

Nous avons donc bien collecté les 10 premiers produits à base de "champagne" de l'API dans le fichier **'berthe_pierrick_5_export_api_122023.csv'** qui a été créé dans le répertoire de travail.

In [10]:
# Afficher temps d'exécution du notebook
calculer_duree_notebook(start_time)

Durée totale pour faire tourner le notebook : 0 minute(s) 2 seconde(s)
